In [3]:
import numpy as np
import random
from scipy.stats import binom

In [50]:
def generate_random_string(length, withsum):
    aux = np.random.rand(length)
    aux = aux/np.sum(aux)
    aux = aux*withsum
    for j in range(length-1):
        aux[j] = float("{:.4f}".format(aux[j]))
    aux[length-1] = withsum-np.sum(aux[:-1])
    return aux

# Balanced server

In [51]:
def generate_balanced_server_input(T, X, B, pa, pb):
    instring = f"{T} {X} {B}\n"
    binomA = binom(X-1, pa)
    binomB = binom(X-1, pb)
    auxA = [binomA.pmf(i) for i in range(X)]
    auxB = [binomB.pmf(i) for i in range(X)]
    for i in range(X-1):
        auxA[i] = float("{:.4f}".format(auxA[i]))
        auxB[i] = float("{:.4f}".format(auxB[i]))
    auxA[X-1] = 1 - np.sum(auxA[:-1])
    auxB[X-1] = 1 - np.sum(auxB[:-1])
    
    for i in range(X):
        instring += "{:.4f}".format(auxA[i]) + ' '
    instring += "\n"
    for i in range(X):
        instring += "{:.4f}".format(auxB[i]) + ' '
    gamma = find_best_gamma(auxA, auxB)
    instring += f"\n{gamma:.4f}"
    return instring
        
    
def imbalance(probsA, probsB, gamma):
    res = 0
    for i in range(len(probsA)):
        for j in range(len(probsB)):
            rew = 0
            if (j - i < gamma):
                rew = +1
            if (j - i > gamma):
                rew = -1
            res += rew*probsA[i]*probsB[j]
    return res

def find_best_gamma(probsA, probsB):
    i = 0
    gamma_u = +len(probsA)
    gamma_l = -len(probsB)
    values = {0:0}
    while i < 100 and max(values.values()) < 10 :
        imb = imbalance(probsA, probsB, (gamma_u + gamma_l)/2)
        if imb not in values:
            values[imb] = 1
        else:
            values[imb] += 1
        max_vals = 0
        if imb < 0:
            gamma_l = (gamma_u + gamma_l)/2
        else:
            gamma_u = (gamma_u + gamma_l)/2
    if np.abs(imbalance(probsA, probsB, gamma_l)) < np.abs(imbalance(probsA, probsB, gamma_u)):
        print(f"accepted imbalance: {imbalance(probsA, probsB, gamma_l)}")
        return gamma_l
    print(f"accepted imbalance: {imbalance(probsA, probsB, gamma_u)}")
    return gamma_u

In [47]:
pastring = f"{pa}".replace('.', '')
pbstring = f"{pb}".replace('.', '')

In [53]:


T = 1000
X = 20
B = 5
pa = 0.51
pb = 0.49

setup = generate_balanced_server_input(T, X, B, pa, pb)

pastring = f"{pa}".replace('.', '')
pbstring = f"{pb}".replace('.', '')
with open(f'inputs/balanced_server_input_T{T}_X{X}_pa{pastring}_pb{pbstring}_B{B}.txt', 'w') as fp:
    fp.write(setup)

accepted imbalance: -0.02974238000000014


# Maximally responsive

In [4]:
n = 10
p = 0.8
rv = binom(n, p)

In [13]:
res = 0
for i in range(11):
    res += rv.pmf(i)

In [23]:
def generate_max_response_input(T, X, B, pa, pb):
    instring = f"{T} {X} {B}\n"
    binomA = binom(X-1, pa)
    binomB = binom(X-1, pb)
    auxA = [binomA.pmf(i) for i in range(X)]
    auxB = [binomB.pmf(i) for i in range(X)]
    for i in range(X-1):
        auxA[i] = float("{:.4f}".format(auxA[i]))
        auxB[i] = float("{:.4f}".format(auxB[i]))
    auxA[X-1] = 1 - np.sum(auxA[:-1])
    auxB[X-1] = 1 - np.sum(auxB[:-1])
    
    for i in range(X):
        instring += "{:.4f}".format(auxA[i]) + ' '
    instring += "\n"
    for i in range(X):
        instring += "{:.4f}".format(auxB[i]) + ' '
    return instring
        
    

In [22]:
T = 10
X = 4
B = 4
pa = 0.3
pb = 0.75

print(generate_max_response_input(T, X, B, pa, pb))

10 4 4
0.3430 0.4410 0.1890 0.0270 
0.0156 0.1406 0.4219 0.4219 


In [ ]:
X = 2
B = 4
pa = 0.3
pb = 0.75
sizes = [1,2,3,4,5,6,7,8,9, 10, 11, 12, 13, 14, 15, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400, 425, 450, 475, 500, 525, 550, 575, 600, 625]
sizes = [1,2,3,4,5,6,7,8,9, 10, 100, 250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 3000, 3250, 3500, 3750, 4000, 4250, 4500, 4750, 5000, 5250, 5500, 5750, 6000]
sizes = [5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000]



for T in sizes:
    setup = generate_max_response_input(T, X, int(B + np.floor(T/10)), pa, pb)
    with open(f'inputs/max_response_input_{T}.txt', 'w') as fp:
        fp.write(setup)


# Stable Clientele

In [6]:
X = 10
B = 16
N = random.sample(range(0, 100), 10)
N = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
sizes = [i for i in range(1,11)] + [20*i for i in range(1,15)]
# sizes = [1,2,3,4,5,6,7,8,9, 10, 20, 50, 100, 200, 500, 1000, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 10000]
sizes = [1,2,3,4,5,6,7,8,9, 10, 100, 250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 3000, 3250, 3500, 3750, 4000, 4250, 4500]
for T in sizes:
    setup = f"{T} {X} {T//3}\n"
#     N = random.sample(range(0, 100), 10)
    aux = generate_random_string(X, 1.0)
    for i in range(len(N)):
        setup += f"{N[i]} "
    setup += "\n"
    for i in range(len(N)):
        setup += "{:.4f}".format(aux[i]) + ' '
    with open(f'inputs/established_clientele_input_{T}.txt', 'w') as fp:
        fp.write(setup)
    
    


In [6]:
def print_list_for_bash(sizes):
    outstr = ""
    for i in range(len(sizes)):
        outstr += f"{sizes[i]} "
    return outstr